<h1 style="text-align:center">Face Detection</h1>
<h3 style="text-align:center">By: <a href="https://www.kaggle.com/rafanthx13">Nancy Galicia</a>, <a href="https://www.kaggle.com/rafaelgreca">Álvaro García</a> and <a href="https://www.kaggle.com/rafaelmm">Omar Sanchez</a></h3>

<h3 >Table of contents</h3>

<div style="margin-top: 20px">
    <ol>
        <li><a href="#introduction">Introduction</a></li>
        <li><a href="#libraries">Libraries</a></li>
        <li><a href="#data">Data</a></li>
        <li><a href="#model">Model</a></li>
        <li><a href="#training">Training</a></li>
        <li><a href="#evaluation">Evaluation</a></li>
        <li><a href="#conclusion">Conclusion</a></li>
    </ol>
</div>

<h2 id="introduction" style="text-align:center">Introduction</h2>

In this notebook we will create a face detection model. We will use the dataset "Labelled Faces in the Wild" (LFW) which contains images of faces of famous people, this is for images that contain faces. For images that do not contain faces, we will use the dataset "Jack", which contains random images. After getting the data, we will label it getting the bounding box of the faces. Then, we will use a pre-trained model to detect the faces in the images. We will use "MobileNetV2" as the pre-trained model. The model will be trained using the dataset we created. The technique we will use is transfer learning. Finally, we will evaluate the model using the test set.

<h2 id="libraries" style="text-align:center">Libraries</h2>

In [1]:
# For data
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import json
# For create the model
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

<h2 id="data" style="text-align:center">Data</h2>

<h3 style="text-align:center">Datasets</h3>

<h4 style="text-align:center">LFW | Jack</h4>

The datasets we will use are the following:

- LFW: This dataset contains images of faces of famous people. It contains 13,233 images of 5,749 people. This dataset is used for images that contain faces.

- Jack: This dataset contains random images. It contains 3,795 images. This dataset is used for images that do not contain faces.

<div style="justify-content:center; display:flex">
    <img src="Dataset/faces/1000_1.jpg" style="width: 250px; height: 250px"/>
    <img src="Dataset/nofaces/1003_0.jpg" style="width: 250px; height: 250px"/>
</div>

<h3 style="text-align:center">Labeled Faces in the Wild</h3>

The dataset "Labeled Faces in the Wild" (LFW) contains images of faces of famous people. This dataset is used for images that contain faces. The dataset contains 13,233 images of 5,749 people. 

We have reduce the size of the dataset to 11,917 images. This was done to perform the quality of the dataset, where we removed the images in which there were more than one face.

The size of each image is 250x250 pixels.

<div style="justify-content:center; display:flex">
    <img src="Dataset/faces/1000_1.jpg" style="width: 250px; height: 250px"/>
    <img src="Dataset/faces/1001_1.jpg" style="width: 250px; height: 250px"/>
    <img src="Dataset/faces/1002_1.jpg" style="width: 250px; height: 250px"/>
    <img src="Dataset/faces/1003_1.jpg" style="width: 250px; height: 250px"/>
</div>


<h3 style="text-align:center">Jack</h3>

The dataset "Jack" contains random images. The dataset contains 3,795 images. This dataset is used for images that do not contain faces, but some images contain faces, but they are not too clear or in a bad position.

We resize the images to 250x250 pixels.

<div style="justify-content:center; display:flex">
    <img src="Dataset/nofaces/1005_0.jpg" style="width: 250px; height: 250px"/>
    <img src="Dataset/nofaces/1007_0.jpg" style="width: 250px; height: 250px"/>
    <img src="Dataset/nofaces/1002_0.jpg" style="width: 250px; height: 250px"/>
    <img src="Dataset/nofaces/1003_0.jpg" style="width: 250px; height: 250px"/>
</div>


<h3 style="text-align:center">Labeling data</h3>
<h4 style="text-align:center">Face | No face</h4>

<div style="justify-content:center; display:flex">
    <img src="Dataset/labeling/labelingDataImage.png" style="width: 250px; height: 250px"/>
    <img src="Dataset/labeling/labelingDataImage0.png" style="width: 250px; height: 250px"/>
</div>

We have labeled the data getting 2 coordinates, the top left and the bottom right of the bounding box of the face. The coordinates are normalized, so they are between 0 and 1. We have created a JSON file with the data of the images. 

The JSON file has the following structure:

```json
{
    "image": "file_name.jpg",
    "bbox": [
        x1, 
        y1,
        x2,
        y2
    ]
    "class": 1|0
}
```
where:
- image: name of the image
- bbox: coordinates of the bounding box (x1, y1, x2, y2), where (x1, y1) is the top left coordinate and (x2, y2) is the bottom right coordinate
- class: 1 if the image contains a face, 0 if the image does not contain a face




<h4 style="text-align:center">Code</h4> 

In [2]:
# This function reads the coordinates of the face from the dataframe previously created and returns them, this csv file contains the coordinates of the face in the image
def get_coordinates(dataframe: pd.DataFrame, image_name: str):
    index = int(image_name.split('.')[0])

    face = dataframe.iloc[index]
    x, y, w, h = face['X'], face['Y'], face['Width'], face['Height']

    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h

    return [x1, y1, x2, y2]

def plot_image(image, coordinates):
    x1, y1, x2, y2 = coordinates
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.imshow(image)
    plt.show()

def createLabel(filename: str, bbox: list, class_id: int, json_file_name: str, img_size: list, json_folder: str ):
    # Convert the bbox to float
    bbox = [float(coord) for coord in bbox]

    # Normalize the bbox coordinates (values between 0 and 1)
    width, height = img_size   

    bbox[0] = bbox[0] / width
    bbox[1] = bbox[1] / height
    bbox[2] = bbox[2] / width
    bbox[3] = bbox[3] / height

    data = {
        "image": filename,
        "bbox": bbox,
        "class": class_id
    }
    if not os.path.exists(json_folder):
        os.makedirs(json_folder)

    with open(f"{json_folder}/{json_file_name}", 'w') as json_file:
        json.dump(data, json_file)

def readLabelPlotImage(json_file_name: str, dataset: str):
    with open(json_file_name, 'r') as json_file:
        data = json.load(json_file)

    image = cv2.imread(f"{dataset}/{data['image']}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    bbox = data['bbox']

    x1 = int(bbox[0] * 250)
    y1 = int(bbox[1] * 250)
    x2 = int(bbox[2] * 250)
    y2 = int(bbox[3] * 250)

    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.imshow(image)
    plt.show()

<h3 style="text-align:center">Split data</h3>

We have split the data into training, validation and test sets. Where the training set contains 80% of the data, the validation set contains 10% of the data and the test set contains 10% of the data.

<lo>
    <li>Training set: 80% of the data</li>
    <li>Validation set: 10% of the data</li>
    <li>Test set: 10% of the data</li>
</lo>

In the split we have taken into account that the data is balanced, that is, the training, validation and test sets contain the same number of images with faces and without faces. So we have around 11000 images for faces and 3000 images for no faces.

<h2 id="model" style="text-align:center">Model</h2>

<h3 style="text-align:center">MobileNetV2</h3>

We will use the pre-trained model "MobileNetV2" to detect the faces in the images. The model is trained on the "ImageNet" dataset. The model has 155 layers and 3,504,872 parameters.

<div style="justify-content:center; display:flex">
    <img src="https://www.researchgate.net/publication/361260658/figure/fig1/AS:1179073011290112@1658124320675/The-architecture-of-MobileNetV2-DNN.png" style="width: 500px; height: 500px"/>
</div>

<br>

<div style="justify-content:center; display:flex">
    <img src="https://www.researchgate.net/publication/342856036/figure/fig3/AS:911929400885251@1594432320422/The-architecture-of-the-MobileNetv2-network.ppm" style="width: 500px; height: 250px"/>
</div>

In [3]:
# Cargar MobileNetV2 preentrenado en ImageNet, excluyendo las capas superiores
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Congelar las capas base
base_model.trainable = False

# Añadir nuevas capas para la clasificación
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Para clasificación binaria

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,570,753 (13.62 MB)

 Trainable params: 1,312,769 (5.01 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

<h2 id="training" style="text-align:center">Training</h2>

<h2 id="evaluation" style="text-align:center">Evaluation</h2>

<h2 id="conclusion" style="text-align:center">Conclusion</h2>